## If you want to follow the procedure by functions, you can run the code below by sequence...

## Load packages

In [1]:
# encoding: utf-8
from selenium import webdriver
import shutil
import time
import logging
import os
import re
import tqdm
from selenium.webdriver.common.by import By
from selenium.webdriver.support import wait, expected_conditions

## Set up param

In [2]:

################ Set up parameters here #####################
default_download_path = "D:/Downloads/"
    # The first string should be the path where your file is downloaded to by default.
    # Most likely, it should be like: "C://Users/usr_name/Downloads"
path = "results/search_1"
query = "TI=(pFind) AND PY=(2016-2022)"
    # These are the task to be searched
driver = webdriver.Chrome(
    executable_path='C://Program Files//Google//Chrome//Application//chromedriver.exe'
    # This is the path where you place your chromedriver
)
#############################################################

if not default_download_path.endswith("/savedrecs.txt"):
    default_download_path += "/savedrecs.txt"
driver.get("https://www.webofscience.com/")

<ipython-input-2-a13ceaf8bf68>:8: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(


## Login in if your ip cannot visit WoS

In [3]:
# Login
try:
    driver.find_element(By.XPATH, '//div[@class="shibboleth-login-form"]')
    input('Login before going next...\n')
except:
    pass


## Close the annoying pop windows

In [4]:
def close_pendo_windows(driver):
    '''Close guiding windows'''
    # Cookies
    try:
        driver.find_element(By.XPATH, '//*[@id="onetrust-accept-btn-handler"]').click()
    except:
        pass
    # "Got it"
    try:
        driver.find_element(By.XPATH, '//button[contains(@class, "_pendo-button-primaryButton")]').click()
    except:
        pass
    # "No thanks"
    try:
        driver.find_element(By.XPATH, '//button[contains(@class, "_pendo-button-secondaryButton")]').click()
    except:
        pass
    # What was it... I forgot...
    try:
        driver.find_element(By.XPATH, '//span[contains(@class, "_pendo-close-guide")').click()
    except:
        pass
    # Overlay
    try:
        driver.find_element(By.XPATH, '//div[contains(@class, "cdk-overlay-container")').click()
    except:
        pass      

## Switch language to English

In [5]:
close_pendo_windows(driver)
try:
    driver.find_element(By.XPATH, '//button[text()[contains(string(), " 简体中文 ")]]').click()
    driver.find_element(By.XPATH, '//button[@lang="en"]').click()
except:
    pass

## Now open the search page and insert the query!
ps: If this block does not run successfully, run it again.

In [7]:
close_pendo_windows(driver)
os.makedirs(path, exist_ok=True)

driver.get("https://www.webofscience.com/wos/alldb/advanced-search")

close_pendo_windows(driver)
# Load the page
wait.WebDriverWait(driver, 10).until(
    expected_conditions.presence_of_element_located((By.XPATH, '//span[contains(@class, "mat-button-wrapper") and text()=" Clear "]')))

# Clear the field
driver.find_element(By.XPATH, '//span[contains(@class, "mat-button-wrapper") and text()=" Clear "]').click()
# Insert the query
driver.find_element(By.XPATH, '//*[@id="advancedSearchInputArea"]').send_keys("{}".format(query))
# Click on the search button
driver.find_element(By.XPATH, '//span[contains(@class, "mat-button-wrapper") and text()=" Search "]').click()

# Wait for the query page
wait.WebDriverWait(driver, 5).until(
    expected_conditions.presence_of_element_located((By.CLASS_NAME, 'title-link')))


<selenium.webdriver.remote.webelement.WebElement (session="e876e94ec439c3d9d6dd26bcde04a290", element="90739b51-98e6-4048-8447-42d80683ea05")>

## Export the search results as outbound
ps: If this block does not run successfully, run it again.

In [23]:
close_pendo_windows(driver)
try:
    driver.find_element(By.XPATH, '//*[contains(@class, "mat-button-wrapper") and text()="Cancel "]').click()
except:
    pass
# Not support search for more than 1000 results yet
assert int(driver.find_element(By.XPATH, '//span[contains(@class, "end-page")]').text) < 1000, "Sorry, too many results!"
# File should not exist on default download folder
assert not os.path.exists(default_download_path), "File existed on default download folder!"

# Click on "Export"          
driver.find_element(By.XPATH, '//span[contains(@class, "mat-button-wrapper") and text()=" Export "]').click()
time.sleep(0.5)
# Click on "Plain text file"  
try:
    driver.find_element(By.XPATH, '//button[contains(@class, "mat-menu-item") and text()=" Plain text file "]').click()
except:
    driver.find_element(By.XPATH, '//button[contains(@class, "mat-menu-item") and @aria-label="Plain text file"]').click()
# Click on "Records from:"
driver.find_element(By.XPATH, '//span[text()[contains(string(), "Records from:")]]').click()
# Click on "Export"
driver.find_element(By.XPATH, '//span[contains(@class, "ng-star-inserted") and text()="Export"]').click()
# Wait for download to complete
for retry_download in range(4):
    time.sleep(1)
    try:
        # If there is any "Internal error"
        wait.WebDriverWait(driver, 2).until(
            expected_conditions.presence_of_element_located((By.XPATH, '//div[text()="Server encountered an internal error"]'))) 
        driver.find_element(By.XPATH, '//div[text()="Server encountered an internal error"]')
        driver.find_element(By.XPATH, '//*[contains(@class, "ng-star-inserted") and text()="Export"]').click()
    except:
        if os.path.exists(default_download_path):
            break
# Download completed
assert os.path.exists(default_download_path), "File not found! Run this block again."

## Move the outbound to dest folder. Now there should be a file at results/search_1/record.txt

In [24]:
shutil.move(default_download_path, os.path.join(path, 'record.txt'))
logging.debug(f'Outbound saved in {path}')
# Load the downloaded outbound (for debug)
with open(os.path.join(path, 'record.txt'), "r", encoding='utf-8') as f_outbound:
    n_record_ref = len(re.findall("\nER\n", f_outbound.read()))
    assert n_record_ref == int("".join(driver.find_element(By.XPATH, '//span[contains(@class, "brand-blue")]').text.split(","))), "Records num do not match outbound num"

## Below are some functions required for the next block

In [25]:

def download_record(driver, path, records_id):
    '''Download a page'''
    # Load the page or throw exception
    wait.WebDriverWait(driver, 10).until(
        expected_conditions.presence_of_element_located((By.XPATH, '//h2[contains(@class, "title")]')))

    # Download the record
    with open(os.path.join(path, f'record-{records_id}.html'), 'w', encoding='utf-8') as file:
        file.write(driver.page_source)
        logging.debug(f'record #{records_id} saved in {path}')

def process_record(driver, path, records_id):
    '''Parse a page'''
    # Show all authors and save raw data
    try:
        driver.find_element(By.XPATH, '//button[text()="...More"]').click()                
    except:
        pass
    with open(os.path.join(path, f'record-{records_id}.dat'), 'w', encoding='utf-8') as file:
        file.write(driver.page_source)
        logging.debug(f'record #{records_id} saved in {path}')  


def process_windows(driver, path, records_id):
    '''Process all subpages'''
    handles = driver.window_handles
    has_error = False
    for i_handle in range(len(driver.window_handles)-1, 0, -1): # traverse in reverse order
        # Switch to the window and load the page
        driver.switch_to.window(handles[i_handle])
        close_pendo_windows(driver)
        try:
            download_record(driver, path, records_id)
            process_record(driver, path, records_id)
        except:
            logging.error("Record downloading failed!")
            has_error = True
        records_id += 1
        driver.close()
    driver.switch_to.window(handles[0])
    return len(handles) - 1 if not has_error else -1

## What are going to do next: click on all the results in new windows, switch to the window and save the page!

In [26]:
# Deal with records
# init
n_record = int(driver.find_element(By.XPATH, '//span[contains(@class, "brand-blue")]').text)
n_page = (n_record + 50 - 1) // 50
assert n_page < 2000, "Too many pages"
logging.debug(f'{n_record} records found, divided into {n_page} pages')

records_id = 0
url_set = set()
for i_page in range(n_page):
    assert len(driver.window_handles) == 1, "Unexpected windows"
    # Roll down to the bottom of the page to show all results
    for i_roll in range(1, 41):
        time.sleep(0.1)
        driver.execute_script(f'window.scrollTo(0, {i_roll * 500});') 
    
    # Open every record in a new window
    windows_count = 0
    for record in driver.find_elements(By.XPATH, '//a[contains(@data-ta, "summary-record-title-link")]'):
        if record.get_attribute("href") in url_set:
            # coz some records have more than 1 href link   
            continue 
        else:
            url_set.add(record.get_attribute("href"))             
        time.sleep(0.5)
        driver.execute_script(f'window.open(\"{record.get_attribute("href")}\");')
        windows_count += 1
        if windows_count >= 10 and not windows_count % 5:
            # Save records and close windows
            records_id += process_windows(driver, path, records_id)
            time.sleep(5)
    
    # Save records and close windows
    records_id += process_windows(driver, path, records_id)
    # Go to the next page
    if i_page + 1 < n_page:            
        driver.find_element(By.XPATH, '//mat-icon[contains(@svgicon, "arrowRight")]').click()

## Now all work are done! Close the driver. That's all.

In [27]:
driver.quit()